In [2]:
import os
import time
import shutil
import telebot
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Configurazione
CARTELLA_MONITORATA = "/percorso/della/cartella/da/monitorare"
FILE_PERCORSI = "percorsi_file.txt"
TOKEN_BOT = "il_tuo_token_bot_telegram"
CHAT_ID = "id_della_chat_telegram"

bot = telebot.TeleBot(TOKEN_BOT)

def salva_percorso(percorso):
    with open(FILE_PERCORSI, "a") as file:
        file.write(percorso + "\n")

def invia_file_telegram(percorso):
    try:
        with open(percorso, "rb") as file:
            bot.send_document(CHAT_ID, file)
        print(f"File inviato con successo: {percorso}")
    except Exception as e:
        print(f"Errore nell'invio del file: {e}")

class NuovoFileHandler(FileSystemEventHandler):
    def on_created(self, event):
        if not event.is_directory:
            percorso = event.src_path
            print(f"Nuovo file rilevato: {percorso}")
            salva_percorso(percorso)
            invia_file_telegram(percorso)

def monitora_cartella():
    event_handler = NuovoFileHandler()
    observer = Observer()
    observer.schedule(event_handler, CARTELLA_MONITORATA, recursive=False)
    observer.start()
    print(f"Monitoraggio della cartella {CARTELLA_MONITORATA} avviato.")
    
    try:
        while True:
            comando = input("Premi 'a' per aggiornare manualmente o 'q' per uscire: ")
            if comando.lower() == 'a':
                percorso = input("Inserisci il percorso del file da inviare: ")
                if os.path.exists(percorso):
                    salva_percorso(percorso)
                    invia_file_telegram(percorso)
                else:
                    print("Percorso non valido.")
            elif comando.lower() == 'q':
                break
    except KeyboardInterrupt:
        pass
    finally:
        observer.stop()
        observer.join()

if __name__ == "__main__":
    monitora_cartella()

FileNotFoundError: [Errno 2] No such file or directory